<a href="https://colab.research.google.com/github/minoooo119/deeplearning_study/blob/main/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98%EB%A1%9C_%EC%8B%9C%EC%9E%91%ED%95%98%EB%8A%94_%EB%94%A5%EB%9F%AC%EB%8B%9D_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤩 다중 선형 회귀 구현 (3-3장)

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [49]:
torch.manual_seed(1)

$H(x)=w_1x_1+w_2x_2+w_3x_3+b$  
처럼 단순 선형 회귀와 다르게 $x$개수가 3개임  
**다중 선형 회귀 (Multivariable Linear Regression)**

## ☑️ 파이토치로 구현하기

In [51]:
# 훈련 데이터
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])


이에 대해서 가중치, 절편 초기화


In [52]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)


가설, 비용 함수, 옵티마이저 선언 후 경사 하강법 1,000회 반복

In [53]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)
#에포크 1번당 경사 하강법 1회
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad() #gradient 누적되기 때문에 매번 초기화 필요
    cost.backward() #w1,w2,w3,b 기준으로 gradient 계산
    optimizer.step() #이후 learning rate 곱하여 기존 가중치와 절편에서 빼준다.

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))


Epoch    0/1000 w1: 0.294 w2: 0.294 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.661 w3: 0.676 b: 0.008 Cost: 1.563628
Epoch  200/1000 w1: 0.679 w2: 0.655 w3: 0.677 b: 0.008 Cost: 1.497595
Epoch  300/1000 w1: 0.684 w2: 0.649 w3: 0.677 b: 0.008 Cost: 1.435044
Epoch  400/1000 w1: 0.689 w2: 0.643 w3: 0.678 b: 0.008 Cost: 1.375726
Epoch  500/1000 w1: 0.694 w2: 0.638 w3: 0.678 b: 0.009 Cost: 1.319507
Epoch  600/1000 w1: 0.699 w2: 0.633 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.627 w3: 0.679 b: 0.009 Cost: 1.215703
Epoch  800/1000 w1: 0.709 w2: 0.622 w3: 0.679 b: 0.009 Cost: 1.167810
Epoch  900/1000 w1: 0.713 w2: 0.617 w3: 0.680 b: 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2: 0.613 w3: 0.680 b: 0.009 Cost: 1.079390


## ☑️ 벡터와 행렬 연산으로 바꾸기
### 1️⃣ 벡터 연산으로
* 위의 계산에서는 x가 3개 존재해서 직접 가정값 계산 가능
* 1000개 이상이라면 힘들기 때문에 벡터 연산으로 진행
$H(x)=w_1x_1+w_2x_2+w_3x_3$
위 식을 벡터의 내적으로 표현

  $\begin{bmatrix}x_1&x_2&x_3\\ \end{bmatrix}⋅\begin{bmatrix}w_1\\w_2\\w_3\\ \end{bmatrix}=\begin{bmatrix}x_1w_1+x_2w_2+x_3w_3\\ \end{bmatrix} $

  로 표현할 수 있음


### 2️⃣ 행렬 연산으로
* y를 결정 짓는데 있어서 총 샘플이 5개라고 하고 특성이 3개가 있다고 하자 이때 가중치와 계산은 벡터의 연산으로 할 수 없음
* 애초에 5×3 행렬과 3×1 행렬의 곱으로 5×1의 결과 y를 결정하게 된다.  

  $\begin{pmatrix}x_{11}&x_{12}&x_{13}\\x_{21}&x_{22}&x_{23}\\x_{31}&x_{32}&x_{33}\\x_{41}&x_{42}&x_{43}\\x_{51}&x_{52}&x_{53}\\\end{pmatrix}$$\begin{pmatrix}w_{1}\\w_{2}\\w_{3}\\\end{pmatrix}$=$\begin{pmatrix}x_{11}w_1+x_{12}w_2+x_{13}w_3\\x_{21}w_1+x_{22}w_2+x_{23}w_3\\x_{31}w_1+x_{32}w_2+x_{33}w_3\\x_{41}w_1+x_{42}w_2+x_{43}w_3\\x_{51}w_1+x_{52}w_2+x_{53}w_3\\ \end{pmatrix}$

  

  * 행렬이용해서 다중 선형 회귀 계산 진행 ⬇️

In [54]:
x_train  =  torch.FloatTensor([[73,  80,  75],
                               [93,  88,  93],
                               [89,  91,  80],
                               [96,  98,  100],
                               [73,  66,  70]])
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])
#특성 3개 5개의 샘플 5*3

In [55]:
print(x_train.shape)
print(y_train.shape)


torch.Size([5, 3])
torch.Size([5, 1])


In [57]:
# 가중치와 편향 선언
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
#가중치는 3*1 벡터임 --> 각 특성에 대해서 가중치가 필요한 거니까 특성 수와 같아야함

In [61]:
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = x_train.matmul(W) + b
    #가중치와 곱해줌
    #위 부분이 행렬을 이용해서 개선된 부분

    # cost 계산
    #가설과 실제 값의 편차 제곱의 평균
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    if epoch%100==0:
      print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
          epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
      ))

Epoch    0/1000 hypothesis: tensor([152.6685, 184.5856, 179.4857, 196.1604, 142.2285]) Cost: 0.192199
Epoch  100/1000 hypothesis: tensor([152.6628, 184.5864, 179.4926, 196.1545, 142.2333]) Cost: 0.189211
Epoch  200/1000 hypothesis: tensor([152.6572, 184.5872, 179.4994, 196.1487, 142.2380]) Cost: 0.186330
Epoch  300/1000 hypothesis: tensor([152.6517, 184.5880, 179.5060, 196.1430, 142.2426]) Cost: 0.183563
Epoch  400/1000 hypothesis: tensor([152.6464, 184.5889, 179.5125, 196.1374, 142.2472]) Cost: 0.180887
Epoch  500/1000 hypothesis: tensor([152.6411, 184.5897, 179.5189, 196.1319, 142.2517]) Cost: 0.178310
Epoch  600/1000 hypothesis: tensor([152.6359, 184.5905, 179.5251, 196.1266, 142.2561]) Cost: 0.175835
Epoch  700/1000 hypothesis: tensor([152.6308, 184.5913, 179.5312, 196.1213, 142.2605]) Cost: 0.173442
Epoch  800/1000 hypothesis: tensor([152.6257, 184.5921, 179.5372, 196.1161, 142.2647]) Cost: 0.171142
Epoch  900/1000 hypothesis: tensor([152.6208, 184.5928, 179.5430, 196.1110, 142.26